In [30]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib
from urllib.request import urlopen
import re
from tqdm import tqdm
import pickle
import concurrent
import numpy as np
import os
import time
from pymongo import MongoClient

In [2]:
client = MongoClient()
db = client.voice

In [3]:
first_stage = list(db.word.find())
ids = [i["_id"] for i in first_stage]

In [9]:
for l in tqdm(first_stage):
    tmp = []

    for i in l['pronounces']:
        tmp.append(db.voices.insert_one(i).inserted_id)

    l['pronounces'] = tmp


100%|██████████| 28642/28642 [00:17<00:00, 1645.95it/s]


In [10]:
first_stage[0]

{'_id': ObjectId('585fbc61698f8208fa0cc408'),
 'metadata': {'added': '2010-04-25', 'num_listenings': '192'},
 'pronounces': [ObjectId('585fda10698f828c848d862d')],
 'word': '0_zero'}

In [12]:
for k in tqdm(first_stage):
    db.words.insert_one(k)

100%|██████████| 28642/28642 [00:10<00:00, 2770.60it/s]


In [27]:
download = lambda path, url: "curl -s -o {0} '{1}' -H 'Accept-Encoding: gzip, deflate, sdch, br' -H 'Accept-Language: ru-RU,ru;q=0.8,en-US;q=0.6,en;q=0.4' -H 'Upgrade-Insecure-Requests: 1' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' -H 'Cookie: forvo_cookies=1; G_ENABLED_IDPS=google; PHPSESSID=ovj7vi6ieo2e6p056ti05rb935; _gat=1; _ga=GA1.2.280471608.1482332671' -H 'Connection: keep-alive' --compressed".format(path, url)

In [ ]:
for item in tqdm(db.voices.find()):
    !{download("mongodb/sounds/%s.mp3" % str(item["_id"]), item["href"])}


0it [00:00, ?it/s]

1it [00:00,  3.99it/s]

2it [00:00,  3.94it/s]

3it [00:00,  3.96it/s]

4it [00:01,  3.83it/s]

5it [00:01,  3.76it/s]

6it [00:01,  3.79it/s]

7it [00:01,  3.86it/s]

8it [00:02,  3.78it/s]

9it [00:02,  3.84it/s]

10it [00:02,  3.90it/s]

11it [00:02,  3.89it/s]

12it [00:03,  3.91it/s]

13it [00:03,  3.86it/s]

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/anaconda3/lib/python3.5/site-packages/tqdm/_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "/anaconda3/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

449it [01:52,  3.89it/s]

In [35]:
ids = [i[:-4] for i in os.listdir("mongodb/sounds")]

In [36]:
for item in tqdm(db.voices.find()):
    if str(item["_id"]) not in ids:
        print("lol")
        break

51223it [00:27, 1849.26it/s]


In [39]:
for _id in tqdm(ids):
    !mpg123 -q -w {"mongodb/sounds/" + _id + ".wav"} {"mongodb/sounds/" + _id + ".mp3"}

100%|██████████| 51223/51223 [1:50:28<00:00,  7.74it/s]


In [40]:
from multiprocessing import Pool

In [42]:
len(os.listdir("mongodb/sounds"))/2

51223.0

In [ ]:
len(ids)